<a href="https://colab.research.google.com/github/asadkarim7/Bioinformatics-Algorithms/blob/master/Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

In [3]:
# Load the sheets into dataframes
reservations_df = pd.read_excel('/content/data.xlsx', sheet_name='reservations')
occupancy_rate_df = pd.read_excel('/content/data.xlsx', sheet_name='occupancy_rate')

In [4]:
# Display the first few rows of each dataframe to understand their structure
reservations_df.head()

,booking_id,booking_date,property_id,status,guest_nationality,arrival_at,departure_at,persons,adults,children,gross_amount,currency,cancelled_at,code,promo_code,room_type
0,CMEMUGWG,2023-07-14 01:47:49.000000 UTC,MAD1,Canceled,ES,2023-07-14,2023-07-15,2,2,0,159.30,EUR,2023-07-14 01:49:52.000000 UTC,BookingCom,NaN,Comfort Suite
1,LQHCJNKX,2023-07-25 06:09:26.000000 UTC,MAD1,Canceled,ES,2023-07-29,2023-07-30,2,2,0,132.21,EUR,2023-07-25 07:07:32.000000 UTC,Ibe,NaN,Comfort Suite
2,ZRDBXBZM,2022-11-09 16:34:00.000000 UTC,MAD1,Canceled,NaN,2022-12-18,2022-12-19,2,2,0,113.30,EUR,2022-12-18 12:02:57.000000 UTC,Direct,NaN,Suite with Sofa Bed
3,OOMOWRXT,2024-05-08 16:00:33.000000 UTC,MAD1,Canceled,ES,2024-05-09,2024-05-10,1,1,0,129.60,EUR,2024-05-09 12:27:21.000000 UTC,BookingCom,NaN,Suite
4,VYMVDICN,2023-01-14 10:06:20.000000 UTC,MAD1,Canceled,ES,2023-03-04,2023-03-05,2,2,0,120.60,EUR,2023-02-24 17:33:23.000000 UTC,BookingCom,NaN,Suite


In [5]:
# Display the first few rows of each dataframe to understand their structure
occupancy_rate_df.head()

,date,occupancy_rate
0,2024-06-10,0.82
1,2024-06-09,1.00
2,2024-06-08,1.00
3,2024-06-07,1.00
4,2024-06-06,0.98


In [6]:
# Filter out canceled reservations
confirmed_reservations_df = reservations_df[reservations_df['status'] != 'Canceled']

In [7]:
reservations_df.shape, confirmed_reservations_df.shape

((21112, 16), (18068, 16))

In [8]:
# Convert date columns to datetime
confirmed_reservations_df['booking_date'] = pd.to_datetime(confirmed_reservations_df['booking_date'])
confirmed_reservations_df['arrival_at'] = pd.to_datetime(confirmed_reservations_df['arrival_at'])
confirmed_reservations_df['departure_at'] = pd.to_datetime(confirmed_reservations_df['departure_at'])

<ipython-input-8-ecbaf8c8e786>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confirmed_reservations_df['booking_date'] = pd.to_datetime(confirmed_reservations_df['booking_date'])
<ipython-input-8-ecbaf8c8e786>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confirmed_reservations_df['arrival_at'] = pd.to_datetime(confirmed_reservations_df['arrival_at'])
<ipython-input-8-ecbaf8c8e786>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [9]:
# Extract the year and month from the booking_date for aggregation purposes
confirmed_reservations_df['year_month'] = confirmed_reservations_df['booking_date'].dt.to_period('M')

<ipython-input-9-368616967ea0>:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  confirmed_reservations_df['year_month'] = confirmed_reservations_df['booking_date'].dt.to_period('M')
<ipython-input-9-368616967ea0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confirmed_reservations_df['year_month'] = confirmed_reservations_df['booking_date'].dt.to_period('M')


In [10]:
# Aggregate data by booking channel and year-month
agg_channel_df = confirmed_reservations_df.groupby(['code', 'year_month']).agg(
    total_bookings=('booking_id', 'count'),
    total_revenue=('gross_amount', 'sum'),
    avg_revenue_per_booking=('gross_amount', 'mean')
).reset_index()

In [11]:
# Display the aggregated data
agg_channel_df

,code,year_month,total_bookings,total_revenue,avg_revenue_per_booking
0,BookingCom,2022-08,83,41693.36,502.329639
1,BookingCom,2022-09,285,116817.85,409.887193
2,BookingCom,2022-10,441,158858.60,360.223583
3,BookingCom,2022-11,557,136306.75,244.715889
4,BookingCom,2022-12,426,81988.70,192.461737
...,...,...,...,...,...
88,Ibe,2024-02,214,50583.21,236.370140
89,Ibe,2024-03,218,67525.38,309.749450
90,Ibe,2024-04,266,70450.42,264.851203
91,Ibe,2024-05,199,56897.81,285.918643


# Forecasting the bookings

In [12]:
# Prepare data for each booking channel
channels = agg_channel_df['code'].unique()
forecast_results = {}

In [14]:
 #Function to prepare data and apply Prophet model
def prepare_and_forecast(channel, agg_channel_df):
    channel_df = agg_channel_df[agg_channel_df['code'] == channel][['year_month', 'total_bookings']]
    channel_df = channel_df.rename(columns={'year_month': 'ds', 'total_bookings': 'y'})
    channel_df['ds'] = channel_df['ds'].dt.to_timestamp()

    # Initialize and fit the Prophet model
    model = Prophet()
    model.fit(channel_df)

    # Create future dataframe for 365 days
    future = model.make_future_dataframe(periods=365)
    forecast = model.predict(future)

    return forecast


In [15]:
# Apply forecasting for each channel
for channel in channels:
    forecast_results[channel] = prepare_and_forecast(channel, agg_channel_df)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 17.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpe0jpmn6_/iutcw4mk.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpe0jpmn6_/moaby7kh.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=82992', 'data', 'file=/tmp/tmpe0jpmn6_/iutcw4mk.json', 'init=/tmp/tmpe0jpmn6_/moaby7kh.json', 'output', 'file=/tmp/tmpe0jpmn6_/prophet_modelvx41smmg/prophet_model-20240620075833.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:58:33 - cmdstanpy - INFO - Chain [1] start processing
I

In [16]:
# Display the forecast results for one channel as an example
forecast_results['BookingCom'].tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
383,2025-05-28,558.815454,351.110509,752.779691,558.815442,558.815466,0.0,0.0,0.0,0.0,0.0,0.0,558.815454
384,2025-05-29,558.920656,375.288852,760.172596,558.920643,558.920668,0.0,0.0,0.0,0.0,0.0,0.0,558.920656
385,2025-05-30,559.025858,375.784801,747.187497,559.025845,559.025870,0.0,0.0,0.0,0.0,0.0,0.0,559.025858
386,2025-05-31,559.131059,367.685409,745.301184,559.131046,559.131072,0.0,0.0,0.0,0.0,0.0,0.0,559.131059
387,2025-06-01,559.236261,352.623744,733.342928,559.236248,559.236273,0.0,0.0,0.0,0.0,0.0,0.0,559.236261
